In [ ]:
import math
import random
import itertools
import datetime
import copy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

def read_edges_from_file(file_name):
    edges = []
    with open(file_name, 'r') as file:
        for line in file:
            node1, node2 = line.strip().split()
            edges.append((str(node1), str(node2)))
    return edges
class DirectGraphInAdjacencyList:
    def __init__(self, edges=[]):
      self.out_neighborhood = {}
      self.in_neighborhood = {}
      self.vertices=set()
      self.build_graph(edges)
      self.highlight_vertices=[]
      self.highlight_edges=[]
      self.active_vertices=set()
        
    def export_networkX_fmt(self):
        tempVertices=list(self.vertices)
        tempEdges=[]
        for v1 in self.out_neighborhood.keys():
            for v2 in self.out_neighborhood[v1]:
                tempEdges.append((v1,v2))
        return tempVertices,tempEdges

    def cut_graph_size(self,n):
        #keep vertices with most neighborhood only
        vertex_scores = {v: len(self.out_neighborhood.get(v, [])) for v in self.vertices}
        sorted_vertices = sorted(vertex_scores.keys(), key=lambda x: vertex_scores[x], reverse=True)
        top_vertices = set(sorted_vertices[:n])   
        
        new_out_neighborhood={}
        for v in top_vertices:
            if v in self.out_neighborhood:
                temp_list=[]
                for out_v in self.out_neighborhood[v]:
                    if out_v in top_vertices:
                        temp_list.append(out_v)
                new_out_neighborhood[v]=temp_list

        self.out_neighborhood=new_out_neighborhood
        self.vertices=top_vertices
        new_in_neighborhood={}
        for v in top_vertices:
            if v in self.in_neighborhood:
                temp_list=[]
                for in_v in self.in_neighborhood[v]:
                    if in_v in top_vertices:
                        temp_list.append(in_v)
                new_in_neighborhood[v]=temp_list
        self.in_neighborhood=new_in_neighborhood

    def build_graph(self, edges):
      for v1,v2 in edges:
        #build the adjacency list for both incoming and outgoing neighborhood
        if v1 not in self.out_neighborhood:
          self.out_neighborhood[v1] = set()
        if v2 not in self.in_neighborhood:
          self.in_neighborhood[v2] = set()
        # Add the edge to the out_neighborhood and in_neighborhood
        self.out_neighborhood[v1].add(v2)
        self.in_neighborhood[v2].add(v1)
        self.vertices.add(v1)
        self.vertices.add(v2)

    def remove_vertice(self, v):
      cnt=0
      if v not in self.active_vertices:
          cnt=1
          self.active_vertices.add(v)
      if v in self.out_neighborhood:
        temp=self.out_neighborhood.pop(v)
        for v2 in temp:
            if v2 not in self.active_vertices:
                self.highlight_edges.append((v,v2))
                self.active_vertices.add(v2)
                cnt+=1
      self.vertices.remove(v)
      self.highlight_vertices.append(v)
      return cnt

    def measure_vertices(self, v):
      cnt=0
      if v not in self.active_vertices:cnt=1
      if v in self.out_neighborhood:
        temp=self.out_neighborhood[v]
        for v2 in temp:
            if v2 not in self.active_vertices:
                cnt+=1
      return cnt

    def get_nodes_influences(self):
      result=[]
      for v in self.vertices:
        influence=self.measure_vertices(v)
        result.append((v,influence))
      return result

    def get_max_from_list(self,node_candidate_list):
      temp_v,max_cnt=node_candidate_list[0]
      for v,cnt in node_candidate_list:
        if cnt>max_cnt:
          temp_v=v
          max_cnt=cnt
      return temp_v

    def get_oracle_max(self,node_candidate_list,lbd,eta):
      right_error_bound=math.log(eta)
      left_error_bound=-math.log(lbd)
      temp_v,max_cnt=node_candidate_list[0][0],0
      for v,cnt in node_candidate_list:
        error_i=random.uniform(left_error_bound,right_error_bound)
        predictError=math.e**error_i
        cnt*=predictError
        if cnt>max_cnt:
          temp_v,max_cnt=v,cnt
      return temp_v

    def get_oracle_single_selection_practical_influence(self,node_candidate_list,lbd,eta):
      right_error_bound=math.log(eta)
      left_error_bound=-math.log(lbd)
      temp_v,max_cnt=node_candidate_list[0][0],0
      practical_result=0
      for v,cnt in node_candidate_list:
        error_i=random.uniform(left_error_bound,right_error_bound)
        predictError=math.e**error_i
        practical_now=cnt
        cnt*=predictError
        if cnt>max_cnt:
          practical_result,max_cnt=practical_now,cnt
      return practical_result

    def greedy_remove_k(self,k):
      result=[]
      if k>len(self.vertices):
        print("wrong k value")
      total_affected=0
      while k>0:
        node_candidate_list=self.get_nodes_influences()
        target_to_remove=self.get_max_from_list(node_candidate_list)
        total_affected+=self.remove_vertice(target_to_remove)
        k-=1
        result.append(total_affected)
      return result

    def greedy_oracle_remove_k(self,k,lmd,eta):
      if k>len(self.vertices):
        print("wrong k value")
      total_affected=0
      result=[]
      while k>0:
        node_candidate_list=self.get_nodes_influences()
        target_to_remove=self.get_oracle_max(node_candidate_list,lmd,eta)
        total_affected+=self.remove_vertice(target_to_remove)
        k-=1
        result.append(total_affected)
      return result

    def random_select_k_vertices(self,k): 
        rdm_k_vertices = random.sample(list(self.vertices),k)
        out_cnt=0
        results=[]
        for v in rdm_k_vertices:
            out_cnt+=self.remove_vertice(v)
            results.append(out_cnt)
        return results

    def random_select_one_based_on_weight(self):
        candidates=list(self.vertices)
        weights=[]
        for v in candidates:
            o_degree=len(self.out_neighborhood.setdefault(v,set()))
            weights.append(max(0,o_degree))
        return random.choices(candidates, weights=weights, k=1)[0]

    def random_k_based_on_weight(self,k):
        out_cnt=0
        results=[]
        for i in range(k):
            out_cnt+=self.remove_vertice(self.random_select_one_based_on_weight())
            results.append(out_cnt)
        return results

    def select_r_vertice_at_once(self,r,lbd,eta):
        vertices_list=list(self.vertices)
        r=min(r,len(vertices_list))
        if r==0:print("no vertice left")
        candidates=list(itertools.combinations(vertices_list, r))
        combination_result=[]
        for ver_lists in candidates:
            new_affected_vertices=set()
            for vertex in ver_lists:
                if vertex not in self.active_vertices:
                    new_affected_vertices.add(vertex)
                if vertex in self.out_neighborhood:
                    for v_affected in self.out_neighborhood[vertex]:
                        if v_affected not in self.active_vertices:
                            new_affected_vertices.add(v_affected)
            combination_result.append((ver_lists,len(new_affected_vertices)))
        oracle_r_vertices=self.get_oracle_max(combination_result,lbd,eta)
        return oracle_r_vertices

    
    def two_step_greedy(self,k,r,lmb,eta):
        out_cnt=0
        counter=0
        results=[]
        while k:
            if k<r:
                r=k
            k-=r
            r_vertices_to_remove=self.select_r_vertice_at_once(r,lmb,eta)
            if r==2:
                base=0
                if len(results):base=results[-1]
                #measure one single selection result 
                results.append(base+self.get_oracle_single_selection_practical_influence(self.get_nodes_influences(),lmb,eta))
            for v in r_vertices_to_remove:
                this_itr=self.remove_vertice(v)
                out_cnt+=this_itr
                counter+=1    
            results.append(out_cnt)
        return results
    
def Influence_Comparison_with_RSOP(filename,k,lbd,eta,scale="normal",times=20,r=2,size=1000):
    def calculate_quartiles(data):
        quartiles = np.percentile(data, [25, 50, 75])
        lower_error = quartiles[0]
        upper_error = quartiles[2]
        return lower_error, upper_error
    def get_errors(sequences):
        lower_errors,upper_errors=[],[]
        for single_x_results in sequences:
            l,u=calculate_quartiles(single_x_results)
            m=np.median(single_x_results)
            lower_errors.append(m-l)
            upper_errors.append(u-m)
        return [lower_errors,upper_errors]   
        
    edges = read_edges_from_file(filename)
    exp_graph=DirectGraphInAdjacencyList(edges)
    exp_graph.cut_graph_size(size)

    greedy_result=[]
    oracle_result=[]
    r_step_result=[]
    rdm_result=[]
    rdm_weight_result=[]

    for _ in range(times):
        greedy_test=copy.deepcopy(exp_graph)
        oracle_test=copy.deepcopy(exp_graph)
        r_step_test=copy.deepcopy(exp_graph)
        rdm_test=copy.deepcopy(exp_graph)
        rdm_weight_test=copy.deepcopy(exp_graph)
        
        single_greedy_result=greedy_test.greedy_remove_k(k)
        single_oracle_result=oracle_test.greedy_oracle_remove_k(k,lbd,eta)
        single_r_step_result=r_step_test.two_step_greedy(k,r,lbd,eta)
        single_rdm_result=rdm_test.random_select_k_vertices(k)
        single_rdm_weight_result=rdm_weight_test.random_k_based_on_weight(k)


        greedy_result.append(single_greedy_result)
        oracle_result.append(single_oracle_result)
        r_step_result.append(single_r_step_result)
        rdm_result.append(single_rdm_result)
        rdm_weight_result.append(single_rdm_weight_result)

    greedy_result=list(zip(*greedy_result))
    oracle_result=list(zip(*oracle_result))
    r_step_result=list(zip(*r_step_result))
    rdm_result=list(zip(*rdm_result))
    rdm_weight_result=list(zip(*rdm_weight_result))
    
    x_values = np.array(list(range(1, len(greedy_result) + 1)))


    colors=['#FF0B0E','#000000','#00FFFF','#FF00FF','#008000']
    markers = ['o', 's', '^', '*', 'D']
    capsize = 5
    alpha = 1
    linewidth = 4
    figsize = (16, 12)

    offset = 0.1  
    
    greedy_medians = [np.median(y) for y in greedy_result]
    oracle_medians = [np.median(y) for y in oracle_result]
    r_step_medians = [np.median(y) for y in r_step_result]
    rdm_medians = [np.median(y) for y in rdm_result]
    rdm_weight_medians = [np.median(y) for y in rdm_weight_result]    
    
    greedy_errors = get_errors(greedy_result)
    oracle_errors = get_errors(oracle_result)
    r_step_errors = get_errors(r_step_result)
    rdm_errors = get_errors(rdm_result)
    rdm_weight_errors = get_errors(rdm_weight_result)
    
    plt.figure(figsize=figsize)
    plt.errorbar(x_values - offset*1.5, greedy_medians, yerr=greedy_errors, label='Greedy',marker=markers[0], capsize=capsize, color=colors[0], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values - offset*0.5, oracle_medians, yerr=oracle_errors, label='SOP-Greedy (ηᵤ=ηₒ={})'.format(lbd),marker=markers[1], capsize=capsize, color=colors[1], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values - offset*0, r_step_medians, yerr=oracle_errors, label='RSOP-Greedy (ηᵤ=ηₒ={})'.format(lbd),marker=markers[2], capsize=capsize, color=colors[2], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values + offset*0.5, rdm_medians, yerr=rdm_errors, label='Random',marker=markers[3], capsize=capsize, color=colors[3], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values + offset*1.5, rdm_weight_medians, yerr=rdm_weight_errors, label='Weighted Random',marker=markers[4], capsize=capsize, color=colors[4], alpha=alpha, linewidth=linewidth)
    
    if scale == "log":plt.yscale('log')
    
    plt.title(filename,fontsize=45)
    plt.xlabel('Number of Selected Nodes (K)',fontsize=45)
    plt.ylabel('Affected Nodes',fontsize=45)
    plt.xticks(fontsize=45)
    plt.yticks(fontsize=45)
    plt.legend(fontsize=32)

    plt.grid(True)
    plt.tight_layout()  
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"plot_{timestamp}.pdf"
    
    plt.savefig(file_name, format='pdf')
    plt.show()

def draw_directed_graph(nodes, directed_edges, highlighted_nodes, active_nodes,highlighted_edges,title,node_amount):
    # Create a directed graph
    G = nx.DiGraph()
    G.add_nodes_from(nodes)
    # Add directed edges to the graph
    G.add_edges_from(directed_edges)
    plt.figure(figsize=(30, 30))

    node_colors = [
        'red' if node in highlighted_nodes 
        else 'orange' if node in active_nodes 
        else 'navy' 
        for node in G.nodes()
    ]
    
    edge_colors = ['tomato' if edge in highlighted_edges else 'grey' for edge in G.edges()]
    node_sizes = [200 for node in G.nodes()]
    edge_widths = [0.4 if edge in highlighted_edges else 0.1 for edge in G.edges()]
    
    # Define positions using spring layout
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, edge_color=edge_colors, width=edge_widths, alpha=0.5, connectionstyle='arc3,rad=0.5')
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, alpha=0.8)  
    plt.scatter([], [], c='red', s=800, label='Selected Nodes ({})'.format(len(highlighted_nodes)))
    plt.scatter([], [], c='orange', s=800, label='Affected Nodes ({})'.format(len(active_nodes)))
    plt.scatter([], [], c='navy', s=800, label='Left Nodes ({})'.format(node_amount - len(highlighted_nodes) - len(active_nodes)))
    plt.legend(loc='best', title=title, fontsize=60, title_fontsize=80)
    plt.margins(0)
    plt.axis('off') 
    plt.tight_layout()  
    file_name = f"plot_{title}.pdf"
    plt.savefig(file_name, format='pdf')
    plt.show()

def Visualise_Influence_Maximization_Result(n_limit,filename,k,r,lbd,eta):
    edges = read_edges_from_file(filename)
    exp_graph=DirectGraphInAdjacencyList(edges)
    exp_graph.cut_graph_size(n_limit)

    nodes,directed_edges=exp_graph.export_networkX_fmt()
    
    greedy_test=copy.deepcopy(exp_graph)
    oracle_test=copy.deepcopy(exp_graph)
    r_step_test=copy.deepcopy(exp_graph)
    rdm_test=copy.deepcopy(exp_graph)
    rdm_weight_test=copy.deepcopy(exp_graph)
    
    single_greedy_result=greedy_test.greedy_remove_k(k)
    single_oracle_result=oracle_test.greedy_oracle_remove_k(k,lbd,eta)
    single_r_step_result=r_step_test.two_step_greedy(k,r,lbd,eta)
    single_rdm_result=rdm_test.random_select_k_vertices(k)
    single_rdm_weight_result=rdm_weight_test.random_k_based_on_weight(k)
    
    # Greedy
    hln_greedy = greedy_test.highlight_vertices
    hle_greedy = greedy_test.highlight_edges
    an_greedy = greedy_test.active_vertices
    G=draw_directed_graph(nodes, directed_edges, hln_greedy, an_greedy, hle_greedy,"Greedy",n_limit)
    
    # Oracle
    hln_oracle = oracle_test.highlight_vertices
    hle_oracle = oracle_test.highlight_edges
    an_oracle = oracle_test.active_vertices
    PG=draw_directed_graph(nodes, directed_edges, hln_oracle, an_oracle, hle_oracle,"SOP-Greedy",n_limit)
    
    # R Step
    hln_r_step = r_step_test.highlight_vertices
    hle_r_step = r_step_test.highlight_edges
    an_r_step = r_step_test.active_vertices
    RPG=draw_directed_graph(nodes, directed_edges, hln_r_step, an_r_step, hle_r_step,"RSOP-Greedy",n_limit)
    
    # Random Weight
    hln_rdm_weight = rdm_weight_test.highlight_vertices
    hle_rdm_weight = rdm_weight_test.highlight_edges
    an_rdm_weight = rdm_weight_test.active_vertices
    WR=draw_directed_graph(nodes, directed_edges, hln_rdm_weight, an_rdm_weight, hle_rdm_weight,"Weighted Random",n_limit)

    # Random
    hln_rdm = rdm_test.highlight_vertices
    hle_rdm = rdm_test.highlight_edges
    an_rdm = rdm_test.active_vertices
    R=draw_directed_graph(nodes, directed_edges, hln_rdm, an_rdm, hle_rdm,"Random",n_limit)

    titles = ["Greedy", "SOP-Greedy", "RSOP-Greedy", "Weighted Random","Random"]
    plt_array=[G,PG,RPG,WR,R]



def Influence_Comparison_Large_Scale_Network(filename,k,lbd,eta,scale="normal",times=20):
    def calculate_quartiles(data):
        quartiles = np.percentile(data, [25, 50, 75])
        lower_error = quartiles[0]
        upper_error = quartiles[2]
        return lower_error, upper_error
    def get_errors(sequences):
        lower_errors,upper_errors=[],[]
        for single_x_results in sequences:
            l,u=calculate_quartiles(single_x_results)
            m=np.median(single_x_results)
            lower_errors.append(m-l)
            upper_errors.append(u-m)
        return [lower_errors,upper_errors]    
        
    edges = read_edges_from_file(filename)
    exp_graph=DirectGraphInAdjacencyList(edges)

    greedy_result=[]
    oracle_result=[]
    oracle_result2=[]
    rdm_result=[]
    rdm_weight_result=[]

    for _ in range(times):
        greedy_test=copy.deepcopy(exp_graph)
        oracle_test=copy.deepcopy(exp_graph)
        oracle_test2=copy.deepcopy(exp_graph)
        rdm_test=copy.deepcopy(exp_graph)
        rdm_weight_test=copy.deepcopy(exp_graph)
        
        single_greedy_result=greedy_test.greedy_remove_k(k)
        single_oracle_result=oracle_test.greedy_oracle_remove_k(k,lbd,eta)
        single_oracle_result2=oracle_test2.greedy_oracle_remove_k(k,2*lbd,2*eta)
        single_rdm_result=rdm_test.random_select_k_vertices(k)
        single_rdm_weight_result=rdm_weight_test.random_k_based_on_weight(k)

        greedy_result.append(single_greedy_result)
        oracle_result.append(single_oracle_result)
        oracle_result2.append(single_oracle_result2)
        rdm_result.append(single_rdm_result)
        rdm_weight_result.append(single_rdm_weight_result)

    greedy_result=list(zip(*greedy_result))
    oracle_result=list(zip(*oracle_result))
    oracle_result2=list(zip(*oracle_result2))
    rdm_result=list(zip(*rdm_result))
    rdm_weight_result=list(zip(*rdm_weight_result))
    
    x_values = np.array(list(range(1, len(greedy_result) + 1)))
    colors=['#FF0B0E','#000000','#00FFFF','#FF00FF','#008000']
    markers = ['o', 's', '^', '*', 'D']
    capsize = 5
    alpha = 1
    linewidth = 4
    figsize = (16,12) 

    offset = 0.1  #

    greedy_medians = [np.median(y) for y in greedy_result]
    oracle_medians = [np.median(y) for y in oracle_result]
    oracle_medians2 = [np.median(y) for y in oracle_result2]
    rdm_medians = [np.median(y) for y in rdm_result]
    rdm_weight_medians = [np.median(y) for y in rdm_weight_result]
    
    greedy_errors = get_errors(greedy_result)
    oracle_errors = get_errors(oracle_result)
    oracle_errors2 = get_errors(oracle_result2)
    rdm_errors = get_errors(rdm_result)
    rdm_weight_errors = get_errors(rdm_weight_result)
  
    plt.figure(figsize=figsize)
    plt.errorbar(x_values - offset*1.5, greedy_medians, yerr=greedy_errors, label='Greedy',marker=markers[0], capsize=capsize, color=colors[0], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values - offset*0.5, oracle_medians, yerr=oracle_errors, label='SOP-Greedy (ηᵤ=ηₒ={})'.format(lbd),marker=markers[1], capsize=capsize, color=colors[1], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values - offset*0, oracle_medians2, yerr=oracle_errors2, label='SOP-Greedy (ηᵤ=ηₒ={})'.format(2*lbd),marker=markers[2], capsize=capsize, color=colors[2], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values + offset*0.5, rdm_medians, yerr=rdm_errors, label='Random',marker=markers[3], capsize=capsize, color=colors[3], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values + offset*1.5, rdm_weight_medians, yerr=rdm_weight_errors, label='Weighted Random',marker=markers[4], capsize=capsize, color=colors[4], alpha=alpha, linewidth=linewidth)
    
    if scale == "log":plt.yscale('log')
    
    plt.title(filename,fontsize=45)
    plt.xlabel('Number of Selected Nodes (K)',fontsize=45)
    plt.ylabel('Affected Nodes',fontsize=45)
    plt.xticks(fontsize=45)
    plt.yticks(fontsize=45)
    plt.legend(fontsize=32)

    plt.grid(True)
    plt.tight_layout()  
      
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"plot_{timestamp}.pdf"
    
    plt.savefig(file_name, format='pdf')
    plt.show()

def Comparison_with_increasing_error(filename,k,lbd,eta,scale="normal",times=20):
    def calculate_quartiles(data):
        quartiles = np.percentile(data, [25, 50, 75])
        lower_error = quartiles[0]
        upper_error = quartiles[2]
        return lower_error, upper_error
    def get_errors(sequences):
        lower_errors,upper_errors=[],[]
        for single_x_results in sequences:
            l,u=calculate_quartiles(single_x_results)
            m=np.median(single_x_results)
            lower_errors.append(m-l)
            upper_errors.append(u-m)
        return [lower_errors,upper_errors]    
        
    edges = read_edges_from_file(filename)
    exp_graph=DirectGraphInAdjacencyList(edges)

    greedy_result=[]
    oracle_result2=[]
    oracle_result4=[]
    oracle_result8=[]
    oracle_result16=[]

    for _ in range(times):
        greedy_test=copy.deepcopy(exp_graph)
        oracle_test2=copy.deepcopy(exp_graph)
        oracle_test4=copy.deepcopy(exp_graph)
        oracle_test8=copy.deepcopy(exp_graph)
        oracle_test16=copy.deepcopy(exp_graph)
        
        single_greedy_result=greedy_test.greedy_remove_k(k)
        single_oracle_result2=oracle_test2.greedy_oracle_remove_k(k,lbd,eta)
        single_oracle_result4=oracle_test4.greedy_oracle_remove_k(k,2*lbd,2*eta)
        single_oracle_result8=oracle_test8.greedy_oracle_remove_k(k,4*lbd,4*eta)
        single_oracle_result16=oracle_test16.greedy_oracle_remove_k(k,8*lbd,8*eta)

        greedy_result.append(single_greedy_result)
        oracle_result2.append(single_oracle_result2)
        oracle_result4.append(single_oracle_result4)
        oracle_result8.append(single_oracle_result8)
        oracle_result16.append(single_oracle_result16)

    greedy_result=list(zip(*greedy_result))
    oracle_result2=list(zip(*oracle_result2))
    oracle_result4=list(zip(*oracle_result4))
    oracle_result8=list(zip(*oracle_result8))
    oracle_result16=list(zip(*oracle_result16))
    
    x_values = np.array(list(range(1, len(greedy_result) + 1)))

    colors = ['#1F78B4', '#008080', '#FFBF00', '#006400', '#6A3D9A']
    markers = ['o', 's', '^', '*', 'D']
    capsize = 5
    alpha = 1
    linewidth = 4
    figsize = (16,12)
    
    offset = 0.1  
    greedy_medians = [np.median(y) for y in greedy_result]
    oracle_medians2 = [np.median(y) for y in oracle_result2]
    oracle_medians4 = [np.median(y) for y in oracle_result4]
    oracle_medians8 = [np.median(y) for y in oracle_result8]
    oracle_medians16 = [np.median(y) for y in oracle_result16]       
    
    greedy_errors = get_errors(greedy_result)
    oracle_errors2 = get_errors(oracle_result2)
    oracle_errors4 = get_errors(oracle_result4)
    oracle_errors8 = get_errors(oracle_result8)
    oracle_errors16 = get_errors(oracle_result16)
  
    plt.figure(figsize=figsize)


    plt.errorbar(x_values - offset*1.5, greedy_medians, yerr=greedy_errors, label='ηᵤ=ηₒ=1',marker=markers[0], capsize=capsize, color=colors[0], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values - offset*0.5, oracle_medians2, yerr=oracle_errors2, label='ηᵤ=ηₒ={}'.format(lbd),marker=markers[1], capsize=capsize, color=colors[1], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values - offset*0, oracle_medians4, yerr=oracle_errors4, label='ηᵤ=ηₒ={}'.format(2*lbd),marker=markers[2], capsize=capsize, color=colors[2], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values + offset*0.5, oracle_medians8, yerr=oracle_errors8, label='ηᵤ=ηₒ={}'.format(4*lbd),marker=markers[3], capsize=capsize, color=colors[3], alpha=alpha, linewidth=linewidth)
    plt.errorbar(x_values + offset*1.5, oracle_medians16, yerr=oracle_errors16, label='ηᵤ=ηₒ={}'.format(8*lbd),marker=markers[4], capsize=capsize, color=colors[4], alpha=alpha, linewidth=linewidth)
    
    if scale == "log": plt.yscale('log')
    
    plt.title(filename,fontsize=45)
    plt.xlabel('Number of Selected Nodes (K)',fontsize=45)
    plt.ylabel('Affected Nodes',fontsize=45)
    plt.xticks(fontsize=45)
    plt.yticks(fontsize=45)
    plt.legend(fontsize=32)

    plt.grid(True)
    plt.tight_layout()  

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"plot_{timestamp}.pdf"
    plt.savefig(file_name, format='pdf')
    plt.show()

In [ ]:
#Performance Comparison on a small scale network, making RSOP-Greedy feasible, R=2 in this experiment
Influence_Comparison_with_RSOP("Facebook_2",30,2,2,scale="log",times=20,r=2,size=1000)
Influence_Comparison_with_RSOP("Facebook_1",30,2,2,scale="log",times=20,r=2,size=1000)
Influence_Comparison_with_RSOP("Twitter",30,2,2,scale="log",times=20,r=2,size=1000)
Influence_Comparison_with_RSOP("Reddit",30,2,2,scale="log",times=20,r=2,size=1000)

#Network Visualisation Experiment
Visualise_Influence_Maximization_Result(160,"reddit",4,2,4,4)

#Compare Performance Under Varying Prediction Errors
Comparison_with_increasing_error("Facebook_1",30,2,2,"log")
Comparison_with_increasing_error("Facebook_2",30,2,2,"log")
Comparison_with_increasing_error("Reddit",30,2,2,"log")
Comparison_with_increasing_error("Twitter",30,2,2,"log")

#Large-scale Experiment
Influence_Comparison_Large_Scale_Network("Facebook_1",30,2,2,"log")
Influence_Comparison_Large_Scale_Network("Facebook_2",30,2,2,"log")
Influence_Comparison_Large_Scale_Network("Reddit",30,2,2,"log")
Influence_Comparison_Large_Scale_Network("Twitter",30,2,2,"log")